In [13]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import xmltodict
from re import search
import datetime


In [2]:
#Create a function takes a url and returns a DataFrame of tournament names and links
def get_link_df(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    a_tags = soup.find_all('a', href = True)
    links_mixed = [link.get('href') for link in a_tags]
    links_fixed = []   
    for link in links_mixed:
        if ('http' in link) == True:
            links_fixed.append(link)
        else:
            links_fixed.append('https://www.pdga.com{}'.format(link))
    names = [name.get_text() for name in a_tags]
    links_df = pd.DataFrame()
    links_df['name'] = names
    links_df['link'] = links_fixed
    event_links_df = links_df[links_df['link'].str.contains("tour/event/")].reset_index(drop=True)
    return event_links_df

In [3]:
#Build a list of urls to iterate over
url_lst = ['https://www.pdga.com/tour/search?date_filter[min][date]=1990-01-01&date_filter[max][date]=2022-11-30&Tier[0]=M&Tier[1]=NT&Classification[0]=Pro&Classification[1]=Pro-Am']
for x in range(1,14):
    url_lst.append('https://www.pdga.com/tour/search?date_filter[min][date]=1990-01-01&date_filter[max][date]=2022-11-30&Tier[0]=M&Tier[1]=NT&Classification[0]=Pro&Classification[1]=Pro-Am&page={}'.format(x))

In [4]:
#Pass each url into get_link_df() and append the result to an empty DataFrame
#event_links_df = pd.DataFrame()
#for url in url_lst:
#    event_links_df = event_links_df.append(get_link_df(url))
event_links_df = pd.read_csv('event_links.csv')
event_links_df

,Unnamed: 0,name,link
0,0,Las Vegas Challenge,https://www.pdga.com/tour/event/47877
1,1,Waco Annual Charity Open,https://www.pdga.com/tour/event/48685
2,2,Jonesboro Open,https://www.pdga.com/tour/event/48567
3,3,OTB Open,https://www.pdga.com/tour/event/48172
4,4,The Preserve Championship,https://www.pdga.com/tour/event/48687
...,...,...,...
328,328,2022 PDGA Professional Disc Golf World Champio...,https://www.pdga.com/tour/event/55460
329,329,2022 PDGA Tim Selinske U.S. Masters Championships,https://www.pdga.com/tour/event/55565
330,330,DGPT - Discraft's Green Mountain Championship,https://www.pdga.com/tour/event/55593
331,331,DGPT - MVP Open at Maple Hill,https://www.pdga.com/tour/event/55594


In [5]:
#Create a function that takes a url and returns a DataFrame of player tournament results
event_links = event_links_df['link'].astype('string')
names = event_links_df['name']

#def get_event_info(url):
#    response = requests.get(url)
#    soup = 

def get_event_df(url):
    r = requests.get(url)
    df = pd.read_html(r.text)[1]
    soup = BeautifulSoup(r.text, 'lxml')
    name = soup.find('h1').text
    date = soup.find('li', 'tournament-date').text.split(':')[1].strip()
    location = soup.find('li', 'tournament-location').text.split(':')[1].strip()
    director = soup.find('li', 'tournament-director').text.split(':')[1].strip()
    df['Event'] = [name] * len(df)
    df['Date Range'] = [date] * len(df)
    df['Location'] = [location] * len(df)
    df['Director'] = [director] * len(df)
    df = df.rename(columns={'Rating': 'Player Rating', 'Unnamed: 7': 'Rd1 Rating', 'Unnamed: 9': 'Rd2 Rating', 'Unnamed: 11': 'Rd3 Rating', 'Unnamed: 13': 'Finals Rating'})
    return df  

In [6]:
#Pass each tournament url into get_event_df() and append the results to an empty DataFrame
events_long = pd.DataFrame()
for url in event_links[:1]:
    events_long = events_long.append(get_event_df(url))
print(events_long)

     Place  Points                 Name   PDGA#  Player Rating  Par  Rd1  \
0        1  1360.0  Eagle Wynne McMahon   37817           1050  -48   48   
1        2  1350.0      Calvin Heimburg   45971           1051  -40   47   
2        2  1350.0        Ezra Aderhold  121715           1016  -40   49   
3        4  1330.0      Garrett Gurthie   13864           1035  -38   49   
4        5  1320.0      Richard Wysocki   38008           1054  -37   49   
..     ...     ...                  ...     ...            ...  ...  ...   
131    132    50.0        Wyatt Merritt   51603            963  +19   61   
132    133    40.0          Chris Bates   35265            964  +21   67   
133    134     0.0      Tristan Lucerne   77246            976  NaN   54   
134    134     0.0        Noah Meintsma   56555           1018  NaN   61   
135    134     0.0        Shea Callahan   36806            995  NaN   54   

     Rd1 Rating  Rd2  Rd2 Rating  Rd3  Rd3 Rating  Finals  Finals Rating  \
0          

In [16]:
#Transform the df from wide form to long form 
#events.to_csv('events.csv')
cols =['Name', 'PDGA#', 'Player Rating', 'Event', 'Date Range', 'Location', 'Director']
vars =['Rd1_combined', 'Rd2_combined', 'Rd3_combined', 'Finals_combined']

#Change the round score and rating into strings and concat them
events_long = events_long.fillna('Cut')
events_long['Rd1'] = events_long['Rd1'].astype('string')
events_long['Rd2'] = events_long['Rd2'].astype('string')
events_long['Rd3'] = events_long['Rd3'].astype('string')
events_long['Finals'] = events_long['Finals'].astype('string')
events_long['Rd1 Rating'] = events_long['Rd1 Rating'].astype('string')
events_long['Rd2 Rating'] = events_long['Rd2 Rating'].astype('string')
events_long['Rd3 Rating'] = events_long['Rd3 Rating'].astype('string')
events_long['Finals Rating'] = events_long['Finals Rating'].astype('string')
events_long['Rd1_combined'] = events_long[['Rd1', 'Rd1 Rating']].agg('-'.join, axis=1)
events_long['Rd2_combined'] = events_long[['Rd2', 'Rd2 Rating']].agg('-'.join, axis=1)
events_long['Rd3_combined'] = events_long[['Rd3', 'Rd3 Rating']].agg('-'.join, axis=1)
events_long['Finals_combined'] = events_long[['Finals', 'Finals Rating']].agg('-'.join, axis=1)

#Melt the df down and split the score and rating back into separate columns
events = pd.melt(events_long, id_vars=cols, value_vars=vars, var_name='combined_cols', value_name='combined_values')
events['Round'] = [x.split('_')[0] for x in events['combined_cols'].values]
events['Throws'] = [x.split('-')[0] for x in events['combined_values'].values]
events['Round Rating'] = [x.split('-')[1] for x in events['combined_values'].values]
events = events[['Name', 'PDGA#', 'Player Rating', 'Round', 'Throws', 'Round Rating', 'Event', 'Date Range', 'Location', 'Director']]
events['First Day'] = [x.split('to')[0].strip() for x in events['Date Range'].values]
events['Last Day'] = [x.split('to')[1] for x in events['Date Range'].values]
events['Year'] = [x.split('-')[2] for x in events['Last Day'].values]
events['First Day'] = events['First Day'].astype('string') + '-' + events['Year'].astype('string')
events['First Day'] = pd.to_datetime(events['First Day'])
events['Last Day'] = pd.to_datetime(events['Last Day'])
events['Event Length'] = events['Last Day'] - events['First Day']
events['# of days not split'] = events['Event Length'].astype('string')
events['# of days less one'] = [int(x.split(' ')[0]) for x in events['# of days not split'].values]
events['# of days'] = events['# of days less one'] + 1
events.loc[events['Round'] == 'Rd1', 'Round'] = '1'
events.loc[events['Round'] == 'Rd2', 'Round'] = '2'
events.loc[events['Round'] == 'Rd3', 'Round'] = '3'
events.loc[events['Round'] == 'Finals', 'Round'] = '4'
date_lst = []
def get_date_lst(day,num):
    lst = [day + datetime.timedelta(days=idx) for idx in range(num)]
    return lst
for day,num in events[['First Day', '# of days']].itertuples(index=False):
    date_lst.append(get_date_lst(day,num))
events['date_lst'] = date_lst
events['Date'] = 

print(events)

                    Name   PDGA#  Player Rating Round Throws Round Rating  \
0    Eagle Wynne McMahon   37817           1050     1     48         1071   
1        Calvin Heimburg   45971           1051     1     47         1080   
2          Ezra Aderhold  121715           1016     1     49         1063   
3        Garrett Gurthie   13864           1035     1     49         1063   
4        Richard Wysocki   38008           1054     1     49         1063   
..                   ...     ...            ...   ...    ...          ...   
539        Wyatt Merritt   51603            963     4    Cut          Cut   
540          Chris Bates   35265            964     4    Cut          Cut   
541      Tristan Lucerne   77246            976     4    Cut          Cut   
542        Noah Meintsma   56555           1018     4    Cut          Cut   
543        Shea Callahan   36806            995     4    Cut          Cut   

                                              Event                   Date 

In [17]:
r = requests.get('https://www.pdga.com/tour/event/48687')
soup = BeautifulSoup(r.text, 'lxml')
#print(soup.prettify())

In [18]:
name = soup.find('h1').text
date = soup.find('li', 'tournament-date').text.split(':')[1].strip()
location = soup.find('li', 'tournament-location').text.split(':')[1].strip()
director = soup.find('li', 'tournament-director').text.split(':')[1].strip()
print(name)
print(date)
print(location)
print(director)


DGPT - The Preserve Championship designed by Leiviska Disc Golf 
30-Jul to 01-Aug-2021
Clearwater, Minnesota, United States
Jeff Spring


In [19]:
#[test_date + datetime.timedelta(days=idx) for idx in range(K)]

In [20]:
print(events['date_lst'][0])

[Timestamp('2021-02-25 00:00:00'), Timestamp('2021-02-26 00:00:00'), Timestamp('2021-02-27 00:00:00'), Timestamp('2021-02-28 00:00:00')]
